# Sistema de Recomendação - Notícias

In [9]:
import pandas as pd
import math
from nltk.tokenize import RegexpTokenizer
import string
import numpy as np

In [10]:
class WordInfo:
    'Classe que guarda informações sobre a palavra, como documentos em que ocorre, tf em cada documento e calculo de idf'

    def __init__(self, word) : # Contrutor da classe recebe apenas a palavra em questão
        self.word = word
        self.idf = 0
        self.docs = {} # Dicionario que mapeia doc_id ao tf da palavra

    '''Incrementa o TF da palavra em um documento'''
    def found(self, doc_id) : # Metodo que realiza a contagem do tf da palavra
        if(doc_id in self.docs) : # Se o doc_id está mapeado, incrementa-o
            self.docs[doc_id] += 1
        else :
            self.docs[doc_id] = 1 # Caso contrario, define-o como 1
            
    def calculateIDF(self, totaldocs) : # Metodo de calculo do idf, recebendo o total de documentos
        df = len(self.docs) # Numero de documentos em que a palavra ocorre 
        if (df > 0) :
            self.idf = math.log(((totaldocs + 1)/df))
            
    def getIds(self) : # Metodo que retorna todos os doc_ids em que a palavra ocorre
        return list(self.docs.keys())
    
    def getTf(self, doc_id) : # Metodo que retorna o tf da palavra em um documento, ou 0 caso não ocorra
        return self.docs.get(doc_id, 0)

## Carregando os dados e selecionando os conjuntos de treinamento e validação

In [11]:
data = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")
data.fillna('', inplace=True) # Preenchendo os campos vazios da tabela com ''
tokenizer = RegexpTokenizer(r'\w+')
data['tokenized_text'] = data.apply(lambda row: tokenizer.tokenize(row['titulo'] + ' ' + row['subTitulo'] + ' ' + row['conteudo']), axis=1)

train = data.sample(frac=0.8)
train.to_csv('tain_set.csv')
validation = data.drop(train.index)
validation.to_csv('validation_set.csv')

In [12]:
vocab = {} # Vocabulario do conjunto de treinamento
docs = {} # Vetores dos documentos

Montagem do vocabulário e vetores dos documentos do conjunto de treinamento

In [13]:
for index, row in train.iterrows() :
    id = row['idNoticia']
    text = row['tokenized_text']
    docs[id] = {}
    for word in text:
        if (word not in string.punctuation):
            word_l = word.lower()
            if (word_l not in vocab) :
                vocab[word_l] = WordInfo(word_l)
            vocab[word_l].found(id)
            docs[id][word_l] = 1
            
# Calculo dos idfs de cada palavra mapeada e atualização dos vetores dos documentos
for word in vocab.keys() : 
    vocab[word].calculateIDF(len(docs))
    idf = vocab[word].idf
    for doc_i in docs.keys() :
        if (word in docs[doc_i]) :
            docs[doc_i][word] = vocab[word].getTf(doc_i) * idf

In [38]:
test = {w:vocab[w].idf for w in vocab.keys()}
np.save('dicts.npy', (docs, test))

d, v = np.load('dicts.npy')

print(v['no'])

0.17335698366910907


In [37]:
v == test

True

# Funções para calcular a similaridade

Calcula a similaridade entre 2 documentos, representados como um vetor, atravez do produto escalar entre os vetores

In [6]:
def similarity(docQ, docI) :
    score = 0
    for word in docQ.keys() :
        score += docQ[word] * docI.get(word, 0)
    return score

Transforma o documento em vetor

In [95]:
def docVect(index) :
    vect = {}
    text = data[data.idNoticia == index].iloc[0]['tokenized_text']
    for word in text:
        if (word not in string.punctuation):
            word_l = word.lower()
            if (word_l not in vect) :
                vect[word_l] = 0
            if (word_l in vocab) :
                vect[word_l] += vocab[word_l].idf
    return vect

Calcula os 5 vizinhos mais próximos de um documento passado através do idNoticia

In [92]:
def top5(index) :
    docQ = docVect(index)
    top = [(id_doc, similarity(docQ, docs[id_doc])) for id_doc in list(docs.keys())[:5]]
    top = sorted(top, reverse=True, key=lambda tup: tup[1])
    for i in list(docs.keys())[5:]:
        sim = similarity(docQ, docs[i])
        j = 4
        while (sim > top[j][1] and j >= 0) :
            j -= 1
        top.insert(j+1, (i, sim))
        top = top[:5]
    return [t[0] for t in top[:5]]

In [ ]:
def get_new(index) :
    doc = {}
    data_doc = data[data.idNoticia == index].iloc[0]
    doc['ID'] = str(data_doc['idNoticia'])
    doc['Titulo'] = str(data_doc['titulo'])
    doc['SubTitulo'] = str(data_doc['subTitulo'])
    doc['Conteudo'] = str(data_doc['conteudo'])[:200] + '...'
    return doc

In [135]:
def get_5_neighbors(index) :
    top = top5(index)
    neighbors = []
    for newID in top:
        doc = get_new(newID)
        neighbors.append(doc)
    return neighbors

Gera um idNoticia aleatório de um dos documentos no conjunto de validação

In [93]:
def randValDoc() :
    return validation.sample(1)['idNoticia'].tolist()[0]

In [114]:
val_doc = randValDoc()
neighbors = top5(val_doc)
print ("Para o documento {} os visinhos mais próximos são: {}".format(val_doc, neighbors))

val_doc = randValDoc()
neighbors = top5(val_doc)
print ("Para o documento {} os visinhos mais próximos são: {}".format(val_doc, neighbors))

val_doc = randValDoc()
neighbors = top5(val_doc)
print ("Para o documento {} os visinhos mais próximos são: {}".format(val_doc, neighbors))

Para o documento 5024 os visinhos mais próximos são: [5024, 7017, 6554, 7, 5129]
Para o documento 5366 os visinhos mais próximos são: [5979, 6099, 5962, 5270, 4479]
Para o documento 3801 os visinhos mais próximos são: [3801, 3942, 7, 2047, 6554]


In [136]:
get_5_neighbors(52)

[{'ID': '52',
  'Titulo': 'Ministério de Dilma: mediocridade é a regra',
  'SubTitulo': 'Na segunda leva de indicados, a presidente prezou mais os laços políticos do que as habilidades técnicas ao escolher um',
  'Conteudo': 'A presidente Dilma Rousseff anunciou ontem treze novos ministros. Somados aos quatro nomes divulgados em novembro, entre os quais os três da equipe econômica, a presidente já definiu 17 dos 39 assento...'},
 {'ID': '7',
  'Titulo': 'Veja os desafios dos governadores que assumem nesta quinta',
  'SubTitulo': '',
  'Conteudo': 'No\xa0Acre, governador reeleito quer erradicar analfabetismo  O governador reeleito Tião Viana (PT) já anunciou que uma das principais metas do segundo mandato é erradicar o analfabetismo. De acordo com ...'},
 {'ID': '54',
  'Titulo': 'Dilma anuncia 13 novos ministros e amplia espaço do PMDB na Esplanada',
  'SubTitulo': 'Para reforçar base no Congresso, presidente mantém loteamento entre partidos aliados, contempla legendas recém-criadas co